## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,74.79,81,47,8.19,scattered clouds
1,1,Batagay,RU,67.6333,134.6333,-21.77,97,100,2.01,overcast clouds
2,2,Hilo,US,19.7297,-155.0900,74.62,75,7,1.01,clear sky
3,3,Cockburn Town,TC,21.4612,-71.1419,79.34,73,98,11.39,overcast clouds
4,4,Belle Fourche,US,44.6714,-103.8522,53.11,38,100,5.55,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp= float(input('What is the minimum temperature you wouuld like for your trip?'))
max_temp= float(input('What is the maximum temperature you wouuld like for your trip?'))

What is the minimum temperature you wouuld like for your trip?75
What is the maximum temperature you wouuld like for your trip?100


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & \
                                      (city_data_df['Max Temp']>=min_temp)]


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             183
City                   183
Country                183
Lat                    183
Lng                    183
Max Temp               183
Humidity               183
Cloudiness             183
Wind Speed             183
Current Description    183
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cockburn Town,TC,79.34,overcast clouds,21.4612,-71.1419,
9,Santa Isabel,BR,78.58,overcast clouds,-23.3156,-46.2214,
11,Bambous Virieux,MU,78.22,few clouds,-20.3428,57.7575,
16,Adelaide,AU,92.16,clear sky,-34.9333,138.6000,
19,Takoradi,GH,79.00,light rain,4.8845,-1.7554,
20,Rikitea,PF,76.44,light rain,-23.1203,-134.9692,
21,Palmerston,AU,96.93,scattered clouds,-12.4860,130.9833,
24,Atuona,PF,79.00,clear sky,-9.8000,-139.0333,
26,Bambanglipuro,ID,83.59,overcast clouds,-7.9500,110.2833,
29,Port Lincoln,AU,81.36,overcast clouds,-34.7333,135.8667,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params['location']=f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url,params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name']=hotels['results'][0]['name']
    except(IndexError):
        print('Hotel not Found...skipping')
    
        

Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()


In [31]:
# 8a. Create the output File (CSV)
output_data_file='WeatherPy_Vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations=hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))